# Genome Assembly Assignment

In this notebook, the functions required to complete **Assignment #6 (Python genome assembly)** are implemented.

**Author:** Elisabetta Roviera (s328422) 

**Course:** Bioquants

**Exame Date:** June 9, 2025  

**Repository:** https://github.com/elisabettaroviera/Bioquants

## Problem 1: Read and Parse Input Data

Write a function `readDataFromFile(fileName)` that reads a file and returns a dictionary mapping read names to sequences.

In [1]:
def readDataFromFile(fileName):
    """
    Reads a file where each line has 'name:sequence' and returns a dict mapping names to sequences.
    """
    reads = {} # Creating the dictionary
    with open(fileName, 'r') as f:
        for line in f:
            name, seq = line.strip().split(':') # Read the file and split in key-value
            reads[name] = seq
    return reads # Returns the dictionary

## Problem 2: Mean Read Length

Write a function `meanLength(fileName)` that returns the mean sequence length as a float.

In [2]:
def meanLength(fileName):
    """
    Calculates the mean length of reads in fileName.
    """
    reads = readDataFromFile(fileName) # Call the function to read the file
    if not reads: # Be sure that the file is read in a proper way
        return "File not read in a proper way"
    total_length = sum(len(seq) for seq in reads.values())
    mean = total_length / len(reads) # Compute the mean
    return mean # Return the value

## Problem 3: Compute Overlap Between Two Reads

Write a function `getOverlap(left, right)` that returns the overlapping substring between the end of `left` and the start of `right`.

In [3]:
def getOverlap(left, right):
    """
    Returns the longest overlap between the 3' end of 'left' and the 5' start of 'right'.
    """
    max_overlap = ''
    max_len = min(len(left), len(right))
    for i in range(max_len, 0, -1):
        if left[-i:] == right[:i]:
            return left[-i:]
    return ''

## Problem 4: Compute All Pairwise Overlaps

Write a function `getAllOverlaps(reads)` that returns a nested dict of overlap lengths for each ordered read pair.

In [4]:
def getAllOverlaps(reads):
    """
    Given a dict of reads, returns a dict of dicts where d[a][b] is 
    the length of the overlap when a is left and b is right.
    """
    overlaps = {} # Create the final dictionary
    for a, seq_a in reads.items():
        overlaps[a] = {}
        for b, seq_b in reads.items():
            if a == b: 
                continue
            overlap_seq = getOverlap(seq_a, seq_b) # Call the function to undersand the overlap
            overlaps[a][b] = len(overlap_seq)
    return overlaps # Return the dictionary

## Problem 5: Pretty-Print Overlap Matrix

Write a function `prettyPrint(overlaps)` that prints an aligned matrix of overlaps with dashes on the diagonal.

In [5]:
def prettyPrint(overlaps):
    """
    Prints a matrix of overlap lengths with '-' on the diagonal.
    """
    names = sorted(overlaps.keys(), key=lambda x: int(x))

    # Header
    print('   ' + ' '.join(f'{name:>3}' for name in names))
    
    for i in names:
        row = [f'{i:>3}']
        for j in names:
            if i == j:
                cell = ' - '
            else:
                cell = f'{overlaps[i].get(j, 0):>3}'
            row.append(cell)
        print(' '.join(row))

## Problem 6: Find the First Read

Write a function `findFirstRead(overlaps)` to identify the read with no significant overlaps (>2) as the first (leftmost).

In [6]:
def findFirstRead(overlaps):
    """
    Returns the name of the read that has no overlaps >2 when it is on the right side.
    """
    reads = overlaps.keys()
    for candidate in reads:
        # Check if no other read has overlap >2 to this candidate
        if all(overlaps[a].get(candidate, 0) <= 2 for a in reads if a != candidate):
            return candidate
    return None

## Problem 7: Determine Read Order

1. Write `findKeyForLargestValue(d)` to find the key with the max value in `d`.
2. Write `findOrder(name, overlaps)` that returns the ordered list of reads.

In [7]:
def findKeyForLargestValue(d):
    """
    Returns the key corresponding to the largest value in dict d.
    """
    return max(d, key=d.get)

def findOrder(name, overlaps):
    """
    Given the first read name and overlaps dict, returns the list of reads in order.
    """
    order = [name]
    current = name

    while True:
        next_overlaps = overlaps[current]
        # Filter only significant overlaps >2
        significant = {r: o for r, o in next_overlaps.items() if o > 2}
        
        if not significant:
            break

        next_read = findKeyForLargestValue(significant)
        order.append(next_read)
        current = next_read
        
    return order

## Problem 8: Assemble Genome

Write `assembleGenome(readOrder, reads, overlaps)` to reconstruct the full genome string.

In [8]:
def assembleGenome(readOrder, reads, overlaps):
    """
    Given the order of reads, the reads dict, and overlaps dict,
    returns the assembled genome sequence.
    """
    genome = reads[readOrder[0]]
    for prev, curr in zip(readOrder, readOrder[1:]):
        o_len = overlaps[prev][curr]
        genome += reads[curr][o_len:]
    return genome

## Example Usage

Load data, compute overlaps, determine order, and assemble genome.

In [9]:
# Problem 1: Read and parse input data
print("### Problem 1: Read Data")
filename = 'genome-assembly.txt'
reads = readDataFromFile(filename)
print(f"reads (names → sequences):\n{reads}")

### Problem 1: Read Data
reads (names → sequences):
{'1': 'GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC', '3': 'GTCTTCAGTAGAAAATTGTTTTTTTCTTCCAAGAGGTCGGAGTCGTGAACACATCAGT', '2': 'CTTTACCCGGAAGAGCGGGACGCTGCCCTGCGCGATTCCAGGCTCCCCACGGG', '5': 'CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC', '4': 'TGCGAGGGAAGTGAAGTATTTGACCCTTTACCCGGAAGAGCG', '6': 'TGACAGTAGATCTCGTCCAGACCCCTAGCTGGTACGTCTTCAGTAGAAAATTGTTTTTTTCTTCCAAGAGGTCGGAGT'}


In [10]:
# Problem 2: Compute mean read length
print("### Problem 2: Mean Read Length")
mean_len = meanLength(filename)
print(f"mean_length: {mean_len:.2f}")

### Problem 2: Mean Read Length
mean_length: 55.33


In [11]:
# Problem 3: Compute overlap between two example reads
print("### Problem 3: Example Overlap")
# using reads '1' and '2' as an example
s1 = reads['1']
s2 = reads['2']
overlap_1_2 = getOverlap(s1, s2)
print(f"getOverlap(read '1', '2') → '{overlap_1_2}' (length: {len(overlap_1_2)})")

### Problem 3: Example Overlap
getOverlap(read '1', '2') → 'C' (length: 1)


In [12]:
# Problem 4: Compute all pairwise overlaps
print("### Problem 4: All Pairwise Overlaps (dict form)")
overlaps = getAllOverlaps(reads)
print(overlaps)

### Problem 4: All Pairwise Overlaps (dict form)
{'1': {'3': 0, '2': 1, '5': 1, '4': 0, '6': 29}, '3': {'1': 0, '2': 0, '5': 0, '4': 1, '6': 1}, '2': {'1': 13, '3': 1, '5': 21, '4': 0, '6': 0}, '5': {'1': 39, '3': 0, '2': 1, '4': 0, '6': 14}, '4': {'1': 1, '3': 1, '2': 17, '5': 2, '6': 0}, '6': {'1': 0, '3': 43, '2': 0, '5': 0, '4': 1}}


In [13]:
# Problem 5: Pretty-print overlap matrix
print("### Problem 5: Overlap Matrix")
prettyPrint(overlaps)

### Problem 5: Overlap Matrix
     1   2   3   4   5   6
  1  -    1   0   0   1  29
  2  13  -    1   0  21   0
  3   0   0  -    1   0   1
  4   1  17   1  -    2   0
  5  39   1   0   0  -   14
  6   0   0  43   1   0  - 


In [14]:
# Problem 6: Find the first (leftmost) read
print("### Problem 6: First Read")
first_read = findFirstRead(overlaps)
print(f"first_read: {first_read}")

### Problem 6: First Read
first_read: 4


In [15]:
# Problem 7: Determine the read order
print("### Problem 7: Read Order")
order = findOrder(first_read, overlaps)
print(f"order: {order}")

### Problem 7: Read Order
order: ['4', '2', '5', '1', '6', '3']


In [16]:
# Problem 8: Assemble the genome
print("### Problem 8: Assembled Genome")
genome_sequence = assembleGenome(order, reads, overlaps)
print("assembled_genome_sequence:")
print(genome_sequence)

### Problem 8: Assembled Genome
assembled_genome_sequence:
TGCGAGGGAAGTGAAGTATTTGACCCTTTACCCGGAAGAGCGGGACGCTGCCCTGCGCGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGCTGGTACGTCTTCAGTAGAAAATTGTTTTTTTCTTCCAAGAGGTCGGAGTCGTGAACACATCAGT
